In [1]:
import pandas as pd
import os
from os.path import dirname

root_path = dirname(os.getcwd())

pd.set_option("display.max_columns", None)
data_dir = root_path + "/data/datasets/original/"
data_dir_processed = root_path + "/data/datasets/processed/"
data_dir_graphs = root_path + "/data/datasets/graphs/"

print(root_path, data_dir, data_dir_processed, data_dir_graphs, sep="\n")

/home/sebdis/ProcessMining/HGNN/HGNN_NA
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/original/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/processed/
/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/datasets/graphs/


In [2]:
dataset = "sp2020"

In [9]:
filename = data_dir + dataset + ".csv"
raw_data = pd.read_csv(filename, index_col=False, sep=";")
raw_data.head()

,CaseID,Activity,time:timestamp,REPAIR_IN_TIME_5D,DEVICETYPE,org:resource
0,Case10,Creation,2018-01-02 13:39:47.0000000,0,AB52,E
1,Case10,Letter,2018-01-05 00:00:00.0000000,0,AB52,E
2,Case10,DeviceReceived,2018-01-05 16:45:34.0000000,0,AB52,E
3,Case10,StockEntry,2018-01-17 00:00:00.0000000,0,AB52,E
4,Case10,InDelivery,2018-01-17 00:00:00.0000000,0,AB52,E


In [10]:
tab_all = raw_data

In [11]:
tab_all["Activity"].unique()

array(['Creation', 'Letter', 'DeviceReceived', 'StockEntry', 'InDelivery',
       'NoteWorkshop', 'Completed', 'NoteHotline', 'StatusRequest',
       'Transmission', 'Approved', 'FreeticketCust', 'FreeticketComp'],
      dtype=object)

In [12]:
tab_all["time:timestamp"] = [x.split(".")[0] for x in tab_all["time:timestamp"]]

tab_all["time:timestamp"] = tab_all["time:timestamp"].str.replace("-", "/")
tab_all["time:timestamp"] = tab_all["time:timestamp"].str.split("+", expand=True)[0]

tab_all.head()

,CaseID,Activity,time:timestamp,REPAIR_IN_TIME_5D,DEVICETYPE,org:resource
0,Case10,Creation,2018/01/02 13:39:47,0,AB52,E
1,Case10,Letter,2018/01/05 00:00:00,0,AB52,E
2,Case10,DeviceReceived,2018/01/05 16:45:34,0,AB52,E
3,Case10,StockEntry,2018/01/17 00:00:00,0,AB52,E
4,Case10,InDelivery,2018/01/17 00:00:00,0,AB52,E


In [13]:
from utils import translate_time

tab_all['time:timestamp'] = tab_all['time:timestamp'].apply(translate_time)
tab_all.head()

,CaseID,Activity,time:timestamp,REPAIR_IN_TIME_5D,DEVICETYPE,org:resource
0,Case10,Creation,1.514897e+09,0,AB52,E
1,Case10,Letter,1.515107e+09,0,AB52,E
2,Case10,DeviceReceived,1.515167e+09,0,AB52,E
3,Case10,StockEntry,1.516144e+09,0,AB52,E
4,Case10,InDelivery,1.516144e+09,0,AB52,E


In [14]:
categorical_columns = ['Activity', "REPAIR_IN_TIME_5D", "DEVICETYPE", "org:resource"]
real_value_columns = ["time:timestamp"]

In [15]:
tab_all.isnull().values.any()

True

In [16]:
tab_all = tab_all.fillna({key: "NAN" for key in categorical_columns})
tab_all = tab_all.fillna({key: -1 for key in real_value_columns})

In [17]:
from math import log


min_time = tab_all['time:timestamp'].min() 

tab_all['time:timestamp'] -= min_time
tab_all['time:timestamp'] = [ log(x)  if x > 0 else 0. for x in tab_all['time:timestamp'].values]


tab_all.head()

,CaseID,Activity,time:timestamp,REPAIR_IN_TIME_5D,DEVICETYPE,org:resource
0,Case10,Creation,18.796956,0,AB52,E
1,Case10,Letter,18.798397,0,AB52,E
2,Case10,DeviceReceived,18.798810,0,AB52,E
3,Case10,StockEntry,18.805478,0,AB52,E
4,Case10,InDelivery,18.805478,0,AB52,E


In [18]:
from utils import get_resource_role_map
roles_map = get_resource_role_map(tab_all)

20.0 Analysing resource pool 
40.0 Analysing resource pool 
60.0 Analysing resource pool 
80.0 Analysing resource pool 
100.0 Analysing resource pool 
('ROLES \n'
 '\n'
 "([{'role': 'Role 1', 'quantity': 12, 'members': ['E', 'G', 'J', 'L', 'NAN', "
 "'C', 'H', 'I', 'K', 'D', 'B', 'A']}], [{'role': 'Role 1', 'resource': 'E'}, "
 "{'role': 'Role 1', 'resource': 'G'}, {'role': 'Role 1', 'resource': 'J'}, "
 "{'role': 'Role 1', 'resource': 'L'}, {'role': 'Role 1', 'resource': 'NAN'}, "
 "{'role': 'Role 1', 'resource': 'C'}, {'role': 'Role 1', 'resource': 'H'}, "
 "{'role': 'Role 1', 'resource': 'I'}, {'role': 'Role 1', 'resource': 'K'}, "
 "{'role': 'Role 1', 'resource': 'D'}, {'role': 'Role 1', 'resource': 'B'}, "
 "{'role': 'Role 1', 'resource': 'A'}])\n"
 '\n')


/home/sebdis/ProcessMining/HGNN/HGNN_NA/data/utils.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['ac_rl'] = self.data.apply(associations, axis=1)


In [19]:
roles_map = {
    x["resource"] : x["role"]
    for x in roles_map
}
roles_map

{'E': 'Role 1',
 'G': 'Role 1',
 'J': 'Role 1',
 'L': 'Role 1',
 'NAN': 'Role 1',
 'C': 'Role 1',
 'H': 'Role 1',
 'I': 'Role 1',
 'K': 'Role 1',
 'D': 'Role 1',
 'B': 'Role 1',
 'A': 'Role 1'}

In [17]:
tab_all["org:resource:role"] = [roles_map[role] for role in tab_all["org:resource"].values]

In [18]:
categorical_columns.append("org:resource:role")

In [20]:
tab_all.to_csv(data_dir_processed + f"{dataset}_processed_all.csv", index=False)

# Split the dataset

In [21]:
split_ratio = 2 / 3

first_act_tab = (
    tab_all.groupby("CaseID").first().sort_values("time:timestamp").reset_index()
)
first_act_tab = first_act_tab[
    ~first_act_tab.duplicated(subset=["CaseID", "Activity"], keep="first")
]
first_act_tab = first_act_tab.reset_index(drop=True)

list_train_valid_cases = list(
    first_act_tab[: int(split_ratio * len(first_act_tab))]["CaseID"].unique()
)

list_train_cases = list_train_valid_cases[: int(len(list_train_valid_cases) * 0.8)]
tab_train = tab_all[tab_all["CaseID"].isin(list_train_cases)].reset_index(drop=True)
tab_train.to_csv(data_dir_processed+ f"{dataset}_processed_train.csv", index = False)

list_valid_cases = list_train_valid_cases[int(len(list_train_valid_cases) * 0.8) :]
tab_valid = tab_all[tab_all["CaseID"].isin(list_valid_cases)].reset_index(drop=True)
tab_valid.to_csv(data_dir_processed+f"{dataset}_processed_valid.csv", index = False)

list_test_cases = list(
    first_act_tab[int(split_ratio * len(first_act_tab)) :]["CaseID"].unique()
)
tab_test = tab_all[tab_all["CaseID"].isin(list_test_cases)].reset_index(drop=True)
tab_test.to_csv(data_dir_processed+ f"{dataset}_processed_test.csv", index = False)

In [22]:
tab_all.columns

Index(['CaseID', 'Activity', 'time:timestamp', 'REPAIR_IN_TIME_5D',
       'DEVICETYPE', 'org:resource'],
      dtype='object')

In [23]:
tab_train

,CaseID,Activity,time:timestamp,REPAIR_IN_TIME_5D,DEVICETYPE,org:resource
0,Case10,Creation,18.796956,0,AB52,E
1,Case10,Letter,18.798397,0,AB52,E
2,Case10,DeviceReceived,18.798810,0,AB52,E
3,Case10,StockEntry,18.805478,0,AB52,E
4,Case10,InDelivery,18.805478,0,AB52,E
...,...,...,...,...,...,...
94105,Case9999,DeviceReceived,18.893535,1,AB34,L
94106,Case9999,Letter,18.894836,1,AB34,L
94107,Case9999,InDelivery,18.895373,1,AB34,L
94108,Case9999,NoteWorkshop,18.895615,1,AB34,L


### Prepare the graphs

In [24]:
from utils import get_case_ids, is_static, get_one_hot_encoder, get_one_hot_encodings

from torch import tensor, max, int64, float32
from torch_geometric.data import HeteroData
import numpy as np

In [25]:
def get_node_features(dataset: pd.DataFrame, trace: pd.DataFrame, cat_features, real_features) -> dict:
    columns_static = [c for c in trace if is_static(trace[c])]

    res = {}

    for key in trace:
        values = trace[key].values
        
        
        if key in cat_features:
            onehot_encoder = get_one_hot_encoder(dataset, key)
            if key not in columns_static:
                res[key] = tensor(
                    get_one_hot_encodings(onehot_encoder, values),
                    dtype=float32,
                    requires_grad=True
                )
            else:
                res[key] = tensor(
                    get_one_hot_encodings(onehot_encoder, np.array([values[0]])),
                    dtype=float32,
                    requires_grad=True
                )
        if key in real_features:
            if key not in columns_static:
                res[key] = tensor(values,  dtype=float32,requires_grad=True)
            else:
                res[key] = tensor([values[0]], dtype=float32,requires_grad=True)
            res[key] = res[key].reshape(res[key].shape[0], 1)
        
    

    return res


In [26]:


def compute_edges_indexs(node_features: dict, prefix_len):
    res = {}
    keys = node_features.keys()
    # indexes = [[i, j] for i in range(prefix_len) for j in range(i + 1, prefix_len)]
    indexes = [[i, i + 1] for i in range(prefix_len-1)]
    # activities indexes
    for k in keys:
        if len(node_features[k]) != 1:
            if k == "Activity":
                res[(k, "followed_by", k)] = indexes
                for k2 in keys:
                    if k2 != k:
                        if len(node_features[k2]) == 1:
                            res[(k, "related_to", k2)] = [
                                [i, 0] for i in range(prefix_len)
                            ]
                        else:
                            res[(k, "related_to", k2)] = [
                                [i, i] for i in range(prefix_len)
                            ]
            else:
                res[(k, "related_to", k)] = indexes

    return res

In [27]:
import torch

from torch import cat

def compute_edges_features(node_features, edges_indexes, cat_features, real_features):
    res = {}

    for k in edges_indexes:
        if k[0] == k[2]:
            indexes = edges_indexes[k]
            res[k] = []
            
            if k[0] in cat_features:
                for i in indexes:
                        res[k].append(      
                            cat(
                                (
                                    tensor([torch.equal(node_features[k[0]][i[0]],node_features[k[0]][i[1]])], dtype=torch.float32),
                                    tensor([i[1] - i[0]], dtype=torch.float32),
                                )
                            )
                        )
            elif k[0] in real_features:
                for i in indexes:
                        res[k].append(
                            tensor(
                                [
                                    node_features[k[0]][i[1]]
                                    - node_features[k[0]][i[0]],
                                    i[1] - i[0],
                                ]
                            )
                        )
            

    return res

In [28]:
# from pprint import pprint as print

from torch import stack

def build_prefixes_graph_from_trace(dataset, trace, cat_features, real_features):
    X = []  # graphs
   

    node_features = get_node_features(dataset, trace, cat_features, real_features)
    
    
    prefix_lenghts = range(2, len(trace))
    # print(prefix_lenghts)
    for prefix in prefix_lenghts:
        # print(prefix)

        # init node types and features
        G = HeteroData()
        for k in node_features:
            G[k].x = node_features[k][:prefix]
        

        edges_indexes = compute_edges_indexs(node_features, prefix)

        edge_features = compute_edges_features(node_features, edges_indexes, cat_features, real_features)
        
        for k in edge_features:
            G[k].edge_attr = stack(edge_features[k])
            


        for k in edges_indexes:
            ce = [[], []]
            for i in range(len(edges_indexes[k])):
                ce[0].append(edges_indexes[k][i][0])
                ce[1].append(edges_indexes[k][i][1])
            edges_indexes[k] = ce

        for k in edges_indexes:
            G[k].edge_index = tensor(edges_indexes[k], dtype=int64)

        G.y = {}
        for k in node_features:
            if k in cat_features:
                G.y[k] = torch.max(node_features[k][0],0)[1] if len(node_features[k]) == 1 else torch.max(node_features[k][prefix], 0)[1]
            else:
                G.y[k] = torch.tensor([node_features[k][prefix][0]])
        
        X.append(G)

        
    return X

## Create the datasets

In [29]:
case_train_ids = get_case_ids(tab_train)
case_valid_ids = get_case_ids(tab_valid)
case_test_ids = get_case_ids(tab_test)

In [30]:
print(len(case_train_ids))
print(len(case_valid_ids))
print(len(case_test_ids))

12749
3188
7969


In [31]:
trace = (
        tab_train.query(f"CaseID == '{case_train_ids[0]}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
trace 

,Activity,time:timestamp,REPAIR_IN_TIME_5D,DEVICETYPE,org:resource
0,Creation,18.796956,0,AB52,E
1,Letter,18.798397,0,AB52,E
2,DeviceReceived,18.798810,0,AB52,E
3,StockEntry,18.805478,0,AB52,E
4,InDelivery,18.805478,0,AB52,E
5,NoteWorkshop,18.805665,0,AB52,E
6,Completed,18.805712,0,AB52,E


In [32]:
graphs = build_prefixes_graph_from_trace(tab_all, trace, categorical_columns, real_value_columns)

In [33]:
graphs[1].y

{'Activity': tensor(11),
 'time:timestamp': tensor([18.8055]),
 'REPAIR_IN_TIME_5D': tensor(0),
 'DEVICETYPE': tensor(41),
 'org:resource': tensor(4)}

In [34]:
graphs[1].x_dict

{'Activity': tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
        grad_fn=<SliceBackward0>),
 'time:timestamp': tensor([[18.7970],
         [18.7984],
         [18.7988]], grad_fn=<SliceBackward0>),
 'REPAIR_IN_TIME_5D': tensor([[1., 0.]], grad_fn=<SliceBackward0>),
 'DEVICETYPE': tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
        grad_fn=<SliceBackward0>),
 'org:resource': tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]],
        grad_fn=<SliceBackward0>)}

In [ ]:
graphs[1].edge_items()

In [35]:
from tqdm.notebook import tqdm


print("Preparing training dataset...")

X_train = []


for i in tqdm(range(len(case_train_ids))):
    trace = (
        tab_train.query(f"CaseID == '{case_train_ids[i]}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    graphs = build_prefixes_graph_from_trace(dataset=tab_all, trace=trace, cat_features=categorical_columns, real_features=real_value_columns)
    
    # print(trace)
    # print([x.x_dict for x in graphs[:2]])
    # print(labels[:2])
    # break

    for i in range(len(graphs)):
        X_train.append(graphs[i])
        

# Y_train = tensor(Y_train, dtype=float32)

print("Done!\n\n")

Preparing training dataset...


  0%|          | 0/12749 [00:00<?, ?it/s]

Done!




In [36]:
print("Preparing validation dataset...")

X_valid = []


for i in tqdm(range(len(case_valid_ids))):
    trace = (
        tab_valid.query(f"CaseID == '{case_valid_ids[i]}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    graphs = build_prefixes_graph_from_trace(dataset=tab_all, trace=trace,cat_features=categorical_columns, real_features=real_value_columns)
    for i in range(len(graphs)):
        X_valid.append(graphs[i])
       

# Y_valid = tensor(Y_valid)

print("Done!\n\n")

Preparing validation dataset...


  0%|          | 0/3188 [00:00<?, ?it/s]

Done!




In [37]:
print("Preparing test dataset...")

X_test = []


for i in tqdm(range(len(case_test_ids))):
    trace = (
        tab_test.query(f"CaseID == '{case_test_ids[i]}'")
        .reset_index()
        .drop(columns="index")
        .drop(columns="CaseID")
    )
    graphs = build_prefixes_graph_from_trace(dataset=tab_all, trace=trace,cat_features=categorical_columns, real_features=real_value_columns)
    for i in range(len(graphs)):
        X_test.append(graphs[i])
     

# Y_test = tensor(Y_test)

print("Done!\n\n")

Preparing test dataset...


  0%|          | 0/7969 [00:00<?, ?it/s]

Done!




### Let's Save the graph datasets

In [38]:
import pickle

with open(data_dir_graphs + dataset + "_TRAIN_event_prediction_FINAL.pkl", "wb") as f:
    pickle.dump(X_train, f)
with open(data_dir_graphs + dataset + "_VALID_event_prediction_FINAL.pkl", "wb") as f:
    pickle.dump(X_valid, f)
with open(data_dir_graphs + dataset + "_TEST_event_prediction_FINAL.pkl", "wb") as f:
    pickle.dump(X_test, f)